<a href="https://colab.research.google.com/github/asia281/dnn2022/blob/main/Asia_of_hw1_checkpoints_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checkpointing

Your task is to implement checkpointing for a MLP using NumPy.

You are free to use the implementation of a MLP and the backpropagation algorithm that you have developed during lab sessions.

The key takeaway from this task is that with checkpointing we can trade off the computational resources needed to compute the forward pass of the network for the memory requirement needed to perform a backward pass in the network, which is often a major bottleneck when training large networks. In plain english, we can slightly increase the time required for training our network to save some of our GPU's precious memory.

## What is checkpointing?

The aim of checkpointing is to save every $n$-th layer's (e.g. every 2-nd layer's) forward result (instead of saving every layer's forward result as in plain backpropagation) and use these checkpoints for recomputing the forward pass of the network upon doing a backward pass. Checkpoint layers are kept in memory after the forward pass, while the remaining activations are recomputed at most once. After being recomputed, the non-checkpoint layers are kept in memory until they are no longer required.

# What should be done

1. Take the implementation a MLP trained with backpropagation. Analyze the algorithm with respect to the memory that is used by the algorithm with respect to the number of hidden layers.

2. Implement a class NetworkWithCheckpointing that inherits from the Network class defined during lab sessions by:
    a) implementing a method `forward_between_checkpoints` that will recompute the forward pass of the network using one of the checkpointed layers
    b) override the method `backprop` to use only checkpointed layers and otherwise compute the activations using `forward_between_checkpoints` method and keep it in memory until no longer needed.

3. Train your network with checkpoinintg on MNIST. Compare running times and memory usage with respect to the network without checkpointing.


# Implement Checkpointing for a MLP

In [124]:
import random
import numpy as np
from torchvision import datasets, transforms

In [125]:
!wget -O mnist.npz https://s3.amazonaws.com/img-datasets/mnist.npz

--2022-11-18 15:37:03--  https://s3.amazonaws.com/img-datasets/mnist.npz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.37, 52.216.136.158, 52.216.49.72, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490434 (11M) [application/octet-stream]
Saving to: ‘mnist.npz’

mnist.npz           100%[===================>]  10.96M  --.-KB/s    in 0.1s    

2022-11-18 15:37:04 (83.5 MB/s) - ‘mnist.npz’ saved [11490434/11490434]



In [126]:
# Let's read the mnist dataset

def load_mnist(path='mnist.npz'):
    with np.load(path) as f:
        x_train, _y_train = f['x_train'], f['y_train']
        x_test, _y_test = f['x_test'], f['y_test']

    x_train = x_train.reshape(-1, 28 * 28) / 255.
    x_test = x_test.reshape(-1, 28 * 28) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1

    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

In [127]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

def softmax(z):
    # Numericaly stable
    exps = np.exp(z - np.max(z))
    sum = np.sum(exps, axis=0)
    res = exps / sum
    return res

In [128]:
class Network(object):
    def __init__(self, sizes, lambda_l2=0.0, momentum=0.0, dropout_p=0.0):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.lambda_l2 = lambda_l2
        self.momentum = momentum
        self.dropout_p = dropout_p
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.weights_momentum = [np.zeros_like(x) for x in self.weights]
        self.biases_momentum = [np.zeros_like(x) for x in self.biases]
    
    def activation(self, z, l):
        if l == (self.num_layers - 1):
            return softmax(z)
        else:
            return sigmoid(z)
    
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for l, b, w in zip(range(1, self.num_layers), self.biases, self.weights):
            if l < self.num_layers - 1:
                a = self.activation((1.0-self.dropout_p)*(np.matmul(w, a)+b), l)
            else:
                a = self.activation(np.matmul(w, a)+b, l)
        return a
    
    def update_mini_batch(self, mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate      
        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)
        
        ### Momentum equation for parameter p
        ### p_(t+1) = p_t + m_(t+1)
        ### m_(t+1) = lambda_momentum * m_t - eta * gradient(p_t)
        self.weights_momentum = [(self.momentum*wm)-(eta/len(mini_batch[0]))*nw 
                                 for wm, w, nw in zip(self.weights_momentum, self.weights, nabla_w)]
        self.biases_momentum = [(self.momentum*bm)-(eta/len(mini_batch[0]))*nb 
                                for bm, b, nb in zip(self.biases_momentum, self.biases, nabla_b)]
                            
        self.weights = [w+wm for w, wm in zip(self.weights, self.weights_momentum)]
        self.biases = [b+bm for b, bm in zip(self.biases, self.biases_momentum)]
        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        ### Dropout
        dropout = [np.random.choice([0,1], size=(k,x.shape[1]), p=[self.dropout_p, 1-self.dropout_p]) for k in self.sizes[:-1]]
        dropout.append(np.full((self.sizes[-1], x.shape[1]), 1))

        g = np.multiply(dropout[0], x)
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for l, b, w in zip(range(1, self.num_layers), self.biases, self.weights):
            f = np.multiply(dropout[l], (np.dot(w, g)+b))
            fs.append(f)
            g = self.activation(f, l)
            gs.append(g)
        # backward pass <- both steps at once
        ### we know that [logloss(softmax)]' ~= [mse]'
        ### change dLdg to dLdf -> considering logloss and softmax as a single block
        ### the rest of dLdf calculations can be left as they were
        dLdf = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for l,w,g in reversed(list(zip(range(1, self.num_layers), self.weights, gs[1:]))):
            if l < (self.num_layers - 1):
                dLdf = np.multiply(np.multiply(dLdg, np.multiply(g,1-g)), dropout[l])
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] 
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] 

        ### L2 regularization
        ### loss = old_loss + lambda_l2 * sum(weights^2)
        dLdWs = [dLdW + (self.lambda_l2 * w) for dLdW, w in zip(dLdWs, self.weights)]

        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))

In [130]:
network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=50, mini_batch_size=100, eta=3., test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.8925
Epoch: 1, Accuracy: 0.9086
Epoch: 2, Accuracy: 0.9187
Epoch: 3, Accuracy: 0.9264
Epoch: 4, Accuracy: 0.9303
Epoch: 5, Accuracy: 0.9349
Epoch: 6, Accuracy: 0.9365
Epoch: 7, Accuracy: 0.9378
Epoch: 8, Accuracy: 0.9402
Epoch: 9, Accuracy: 0.9421
Epoch: 10, Accuracy: 0.9441
Epoch: 11, Accuracy: 0.9438
Epoch: 12, Accuracy: 0.9442
Epoch: 13, Accuracy: 0.9449
Epoch: 14, Accuracy: 0.9452
Epoch: 15, Accuracy: 0.9463
Epoch: 16, Accuracy: 0.9472
Epoch: 17, Accuracy: 0.9473
Epoch: 18, Accuracy: 0.9469
Epoch: 19, Accuracy: 0.947
Epoch: 20, Accuracy: 0.9468
Epoch: 21, Accuracy: 0.9465
Epoch: 22, Accuracy: 0.9474
Epoch: 23, Accuracy: 0.9489
Epoch: 24, Accuracy: 0.9488
Epoch: 25, Accuracy: 0.9471
Epoch: 26, Accuracy: 0.9467
Epoch: 27, Accuracy: 0.9476
Epoch: 28, Accuracy: 0.9484
Epoch: 29, Accuracy: 0.9487
Epoch: 30, Accuracy: 0.9484
Epoch: 31, Accuracy: 0.9481
Epoch: 32, Accuracy: 0.9479
Epoch: 33, Accuracy: 0.948
Epoch: 34, Accuracy: 0.9479
Epoch: 35, Accuracy: 0.9487
Epoc

In [131]:
!pip install -U memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [154]:
class NetworkWithCheckpointing(Network):
    def __init__(self, sizes, checkpoint_every_nth_layer: int = 0, *args, **kwargs):
        super().__init__(sizes, *args, **kwargs)
        self.checkpoint_freq = checkpoint_every_nth_layer
        self.layers_nr = len(sizes)

    def get(self, d: dict, idx: int):
      if idx in d:
        return d[idx]
      d[idx] = sigmoid(np.matmul(self.weights[idx - 1], self.get(d, idx - 1)) + self.biases[idx - 1])
      return d[idx]

    def delete(self, d: dict, idx: int):
      if idx % self.checkpoint_freq != 0 and idx in d:
        del d[idx]

    def forward_between_checkpoints(self, a, start, end):
      is_last = (end == self.layers_nr - 1)
      if is_last:
        end -= 1
      for b, w in zip(self.biases[start:end], self.weights[start:end]):
        a = sigmoid(np.dot(w, a) + b)   
      if is_last:
        return softmax(np.matmul(self.weights[-1], a) + self.biases[-1])

      return a

    def backprop(self, x, y):
      gs = {}
      g = x
      gs[0] = g

      for i in range(0, self.layers_nr - 1, self.checkpoint_freq):
        end = min(self.layers_nr - 1, i + self.checkpoint_freq)
        g = self.forward_between_checkpoints(g, i, end)
        gs[end] = g
      # backward pass <- both steps at once
      idx = self.layers_nr - 1
      dLdf = self.cost_derivative(gs[idx], y)
      dLdfs = []
      dLdWs = []
      for idx, w in reversed(list(zip(range(1, self.num_layers), self.weights))):
        if idx < (self.num_layers - 1):
          g = self.get(gs, idx)
          self.delete(gs, idx)
          dLdf = np.multiply(dLdg,np.multiply(g,1-g))

        dLdfs.append(dLdf)
        dLdg = np.matmul(w.T, dLdf)
      
      for idx, w in enumerate(reversed(dLdfs)):
        dLdWs.append(np.matmul(w, self.get(gs, idx).T))
      
      dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] 
      return (dLdBs, dLdWs)

        


In [147]:
from memory_profiler import memory_usage
%load_ext memory_profiler

%time %memit NetworkWithCheckpointing([784,30,10], 2).SGD((x_train, y_train), epochs=50, mini_batch_size=100, eta=3., test_data=(x_test, y_test))

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Epoch: 0, Accuracy: 0.8915
Epoch: 1, Accuracy: 0.9138
Epoch: 2, Accuracy: 0.923
Epoch: 3, Accuracy: 0.9274
Epoch: 4, Accuracy: 0.9322
Epoch: 5, Accuracy: 0.9363
Epoch: 6, Accuracy: 0.9374
Epoch: 7, Accuracy: 0.9393
Epoch: 8, Accuracy: 0.9389
Epoch: 9, Accuracy: 0.9407
Epoch: 10, Accuracy: 0.943
Epoch: 11, Accuracy: 0.9447
Epoch: 12, Accuracy: 0.9455
Epoch: 13, Accuracy: 0.9471
Epoch: 14, Accuracy: 0.9476
Epoch: 15, Accuracy: 0.9482
Epoch: 16, Accuracy: 0.9488
Epoch: 17, Accuracy: 0.948
Epoch: 18, Accuracy: 0.948
Epoch: 19, Accuracy: 0.9476
Epoch: 20, Accuracy: 0.9473
Epoch: 21, Accuracy: 0.9479
Epoch: 22, Accuracy: 0.9482
Epoch: 23, Accuracy: 0.9479
Epoch: 24, Accuracy: 0.9476
Epoch: 25, Accuracy: 0.947
Epoch: 26, Accuracy: 0.9477
Epoch: 27, Accuracy: 0.9472
Epoch: 28, Accuracy: 0.9475
Epoch: 29, Accuracy: 0.947
Epoch: 30, Accuracy: 0.9461
Epoch: 31, Accuracy: 0.9463
Epoch: 32, Accuracy: 0

Overwriting mprun_demo.py
